In [2]:
# test.py
import sqlite3
import chess
import chess.pgn
import io
from features.extractor import FeatureExtractor
from analysis.stockfish_handler import StockfishHandler

# Connect to database
DB_PATH = "../data/processed/chess_games.db"
conn = sqlite3.connect(DB_PATH)

# Get first game
query = """
SELECT 
    moves,
    white, black,
    white_elo, black_elo,
    eco, opening,
    result
FROM chess_games 
WHERE moves IS NOT NULL
  AND white_elo IS NOT NULL
  AND black_elo IS NOT NULL
LIMIT 1
"""


row = conn.execute(query).fetchone()

# Print game info
print("Game Information:")
print("-" * 40)
print(f"White: {row[1]} ({row[3]})")
print(f"Black: {row[2]} ({row[4]})")
print(f"ECO: {row[5]}")
print(f"Opening: {row[6]}")
print(f"Result: {row[7]}")
print("\nPGN:")
print(row[0])

# # change row 0 to cut moves till 8th move regex 8.
# moves = row[0].split(" ")[0:22]
# row = list(row)
# row[0] = " ".join(moves)
# row = tuple(row)
# print(row[0])

# Parse game
game = chess.pgn.read_game(io.StringIO(row[0]))

# Initialize feature extractor
feature_extractor = FeatureExtractor()

# Extract features without engine first
# print("\nFeatures without engine analysis:")
# print("-" * 40)
# features = feature_extractor.extract_features(game)
# for name, value in features.__dict__.items():
#     print(f"{name}: {value:.3f}")

# Initialize stockfish and get evaluations
stockfish = StockfishHandler(path="stockfish", depth=20)  # Update path
try:
    # Get positions and engine evaluations
    positions = feature_extractor._get_positions(game)
    print("\nPositions:")
    print(positions)
    print('-'*40)
    evals = [stockfish.evaluate_position(pos, i) for i, pos in enumerate(positions)]
    print("\nEngine evaluations:")
    print(evals)

    # Extract features with engine
    print("\nFeatures with engine analysis:")
    print("-" * 40)
    features_with_engine = feature_extractor.extract_features(game, evals)
    for name, value in features_with_engine.__dict__.items():
        print(f"{name}: {value:.3f}")
finally:
    stockfish.close()

conn.close()

Game Information:
----------------------------------------
White: Abdallah1990 (2510)
Black: EzWin78 (2647)
ECO: D00b
Opening: None
Result: 0-1

PGN:
1.d4 d5 2.c3 Nf6 3.c4 e6 4.Nf3 Bb4+ 5.Bd2 Be7 6.Nc3 O-O 7.cxd5 exd5 8.Bg5 b6 9.e3 Bb7 10.Bd3 Nbd7 11.O-O c5 12.Rc1 c4 13.Bb1 a6 14.Ne5 b5 15.a3 Nxe5 16.dxe5 Nd7 17.Bxe7 Qxe7 18.f4 Rad8 19.Nxd5 Qc5 20.Be4 Bxd5 21.Qxd5 Qxe3+ 22.Kh1 Nf6 23.exf6 Rxd5 24.fxg7 Rfd8 25.Bxd5 Rxd5 26.f5 Kxg7 27.f6+ Kg6 { Normal} 0-1

Positions:
[Board('rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'), Board('rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1'), Board('rnbqkbnr/ppp1pppp/8/3p4/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 0 2'), Board('rnbqkbnr/ppp1pppp/8/3p4/3P4/2P5/PP2PPPP/RNBQKBNR b KQkq - 0 2'), Board('rnbqkb1r/ppp1pppp/5n2/3p4/3P4/2P5/PP2PPPP/RNBQKBNR w KQkq - 1 3'), Board('rnbqkb1r/ppp1pppp/5n2/3p4/2PP4/8/PP2PPPP/RNBQKBNR b KQkq - 0 3'), Board('rnbqkb1r/ppp2ppp/4pn2/3p4/2PP4/8/PP2PPPP/RNBQKBNR w KQkq - 0 4'), Board('rnbqkb1r/ppp2ppp/4pn2/

In [1]:
# test_moves.py
import chess
import chess.pgn
import io
from features.extractor import FeatureExtractor
from analysis.stockfish_handler import StockfishHandler
from tqdm import tqdm

# Moves input as a string (PGN format)
moves = """
1. e4 d6 2. d4 Nf6 3. Nc3 g6 { B07 Pirc Defense } 4. Be3 Bg7 5. Qd2 c6 6. f3 b5 7. Nge2 Nbd7 8. Bh6 Bxh6 9. Qxh6 Bb7 10. a3 e5 11. O-O-O Qe7 12. Kb1 a6 13. Nc1 O-O-O 14. Nb3 exd4 15. Rxd4 c5 16. Rd1 Nb6 17. g3 Kb8 18. Na5 Ba8 19. Bh3 d5 20. Qf4+ Ka7 21. Rhe1 d4 22. Nd5 Nbxd5 23. exd5 Qd6 24. Rxd4 cxd4?? { (-0.42 → 2.64) Blunder. Kb6 was best. } (24... Kb6) 25. Re7+ Kb6 26. Qxd4+ Kxa5 27. b4+ Ka4 28. Qc3 Qxd5 29. Ra7 Bb7 30. Rxb7 Qc4 31. Qxf6 Kxa3? { (0.73 → 2.35) Mistake. Rd1+ was best. } (31... Rd1+ 32. Kb2 Ra8 33. Qb6 Qd4+ 34. Qxd4 Rxd4 35. Rxf7 a5 36. Be6 axb4 37. Bb3+) 32. Qxa6+ Kxb4 33. c3+ Kxc3 34. Qa1+ Kd2 35. Qb2+ Kd1 36. Bf1 Rd2 37. Rd7 Rxd7 38. Bxc4 bxc4 39. Qxh8 Rd3 40. Qa8 c3?! { (2.61 → 3.63) Inaccuracy. Rb3+ was best. } (40... Rb3+ 41. Ka2 Rd3 42. Qa4+ Ke1 43. Qxc4 Rxf3 44. g4 Kf2 45. g5 Ke1 46. Qg4) 41. Qa4+ Ke1 42. f4 f5?! { (3.29 → 4.47) Inaccuracy. Rf3 was best. } (42... Rf3 43. Kc2 Rf2+ 44. Kxc3 Rxh2 45. g4 h5 46. g5 Rg2 47. Qe8+ Kf2 48. Qxf7) 43. Kc1 Rd2 44. Qa7 { White wins. } 1-0"""
# Create a file-like object from the moves string
pgn_io = io.StringIO(moves)

# Parse the game from the PGN string
game = chess.pgn.read_game(pgn_io)
if game is None:
    print("Failed to parse game!")
else:
    print("Game parsed successfully!\n")

# Print the PGN for verification
print("PGN:")
print(moves)
print("\n")

# Initialize the feature extractor and the Stockfish handler
feature_extractor = FeatureExtractor()
stockfish = StockfishHandler(path="stockfish", depth=20)

try:
    # Get positions from the game using the extractor method
    positions = feature_extractor._get_positions(game)
    # print("Positions:")
    # for i, board in enumerate(positions):
    #     print(f"Move {i}: {board.fen()}")
    # print("-" * 40)

    # Evaluate each position using Stockfish
    evals = [stockfish.evaluate_position(pos, i) for i, pos in tqdm(enumerate(positions), total=len(positions), desc="Evaluating positions")]
    # print("\nEngine evaluations:")
    # for i, info in enumerate(evals):
    #     print(f"Ply {i}: {info.eval}")
    # print("-" * 40)

    # Extract features using both the game and engine evaluations
    features_with_engine = feature_extractor.extract_features(game, evals)
    print("\nFeatures with engine analysis:")
    for name, value in features_with_engine.__dict__.items():
        print(f"{name}: {value:.3f}")
finally:
    stockfish.close()


Game parsed successfully!

PGN:

1. e4 d6 2. d4 Nf6 3. Nc3 g6 { B07 Pirc Defense } 4. Be3 Bg7 5. Qd2 c6 6. f3 b5 7. Nge2 Nbd7 8. Bh6 Bxh6 9. Qxh6 Bb7 10. a3 e5 11. O-O-O Qe7 12. Kb1 a6 13. Nc1 O-O-O 14. Nb3 exd4 15. Rxd4 c5 16. Rd1 Nb6 17. g3 Kb8 18. Na5 Ba8 19. Bh3 d5 20. Qf4+ Ka7 21. Rhe1 d4 22. Nd5 Nbxd5 23. exd5 Qd6 24. Rxd4 cxd4?? { (-0.42 → 2.64) Blunder. Kb6 was best. } (24... Kb6) 25. Re7+ Kb6 26. Qxd4+ Kxa5 27. b4+ Ka4 28. Qc3 Qxd5 29. Ra7 Bb7 30. Rxb7 Qc4 31. Qxf6 Kxa3? { (0.73 → 2.35) Mistake. Rd1+ was best. } (31... Rd1+ 32. Kb2 Ra8 33. Qb6 Qd4+ 34. Qxd4 Rxd4 35. Rxf7 a5 36. Be6 axb4 37. Bb3+) 32. Qxa6+ Kxb4 33. c3+ Kxc3 34. Qa1+ Kd2 35. Qb2+ Kd1 36. Bf1 Rd2 37. Rd7 Rxd7 38. Bxc4 bxc4 39. Qxh8 Rd3 40. Qa8 c3?! { (2.61 → 3.63) Inaccuracy. Rb3+ was best. } (40... Rb3+ 41. Ka2 Rd3 42. Qa4+ Ke1 43. Qxc4 Rxf3 44. g4 Kf2 45. g5 Ke1 46. Qg4) 41. Qa4+ Ke1 42. f4 f5?! { (3.29 → 4.47) Inaccuracy. Rf3 was best. } (42... Rf3 43. Kc2 Rf2+ 44. Kxc3 Rxh2 45. g4 h5 46. g5 Rg2 47. Qe8+ Kf2 

Evaluating positions: 100%|██████████| 88/88 [02:24<00:00,  1.64s/it]

+0.3 -> +0.3 = Judgment.GOOD
+0.3 -> +0.6 = Judgment.GOOD
+0.6 -> +0.5 = Judgment.GOOD
+0.5 -> +0.5 = Judgment.GOOD
+0.5 -> +0.6 = Judgment.GOOD
+0.6 -> +0.8 = Judgment.GOOD
+0.8 -> +0.7 = Judgment.GOOD
+0.7 -> +0.8 = Judgment.GOOD
+0.8 -> +0.8 = Judgment.GOOD
+0.8 -> +0.8 = Judgment.GOOD
+0.8 -> +0.4 = Judgment.GOOD
+0.4 -> +0.4 = Judgment.GOOD
+0.4 -> +0.4 = Judgment.GOOD
+0.4 -> +0.4 = Judgment.GOOD
+0.4 -> +0.3 = Judgment.GOOD
+0.3 -> +0.3 = Judgment.GOOD
+0.3 -> +0.3 = Judgment.GOOD
+0.3 -> +0.2 = Judgment.GOOD
+0.2 -> +0.2 = Judgment.GOOD
+0.2 -> +0.4 = Judgment.GOOD
+0.4 -> +0.4 = Judgment.GOOD
+0.4 -> +0.5 = Judgment.GOOD
+0.5 -> +0.4 = Judgment.GOOD
+0.4 -> +0.8 = Judgment.GOOD
+0.8 -> +0.5 = Judgment.GOOD
+0.5 -> +0.7 = Judgment.GOOD
+0.7 -> +0.5 = Judgment.GOOD
+0.5 -> +0.3 = Judgment.GOOD
+0.3 -> +0.4 = Judgment.GOOD
+0.4 -> +0.4 = Judgment.GOOD
+0.4 -> +0.4 = Judgment.GOOD
+0.4 -> +0.3 = Judgment.GOOD
+0.3 -> +0.2 = Judgment.GOOD
+0.2 -> +0.3 = Judgment.GOOD
+0.3 -> +0.0 =

In [1]:
import chess
import chess.pgn
import io
from features.extractor import FeatureExtractor
from analysis.stockfish_handler import StockfishHandler
from analysis.move_analyzer import MoveAnalyzer
from tqdm import tqdm
from models.enums import Judgment
import pandas as pd
from tabulate import tabulate  # You may need to install this: pip install tabulate
import colorama  # You may need to install this: pip install colorama
from colorama import Fore, Style

# Initialize colorama
colorama.init(autoreset=True)

# Color mapping for move judgments
JUDGMENT_COLORS = {
    Judgment.BRILLIANT: Fore.MAGENTA,   # Magenta for brilliant moves
    Judgment.GREAT: Fore.CYAN,          # Cyan for great moves
    Judgment.GOOD: Fore.GREEN,          # Green for good moves
    Judgment.INACCURACY: Fore.YELLOW,   # Yellow for inaccuracies
    Judgment.MISTAKE: Fore.RED,         # Red for mistakes
    Judgment.BLUNDER: Fore.RED + Style.BRIGHT  # Bright red for blunders
}

# Moves input as a string (PGN format)
moves = """
1. e4 d6 2. d4 Nf6 3. Nc3 g6 { B07 Pirc Defense } 4. Be3 Bg7 5. Qd2 c6 6. f3 b5 7. Nge2 Nbd7 8. Bh6 Bxh6 9. Qxh6 Bb7 10. a3 e5 11. O-O-O Qe7 12. Kb1 a6 13. Nc1 O-O-O 14. Nb3 exd4 15. Rxd4 c5 16. Rd1 Nb6 17. g3 Kb8 18. Na5 Ba8 19. Bh3 d5 20. Qf4+ Ka7 21. Rhe1 d4 22. Nd5 Nbxd5 23. exd5 Qd6 24. Rxd4 cxd4?? { (-0.42 → 2.64) Blunder. Kb6 was best. } (24... Kb6) 25. Re7+ Kb6 26. Qxd4+ Kxa5 27. b4+ Ka4 28. Qc3 Qxd5 29. Ra7 Bb7 30. Rxb7 Qc4 31. Qxf6 Kxa3? { (0.73 → 2.35) Mistake. Rd1+ was best. } (31... Rd1+ 32. Kb2 Ra8 33. Qb6 Qd4+ 34. Qxd4 Rxd4 35. Rxf7 a5 36. Be6 axb4 37. Bb3+) 32. Qxa6+ Kxb4 33. c3+ Kxc3 34. Qa1+ Kd2 35. Qb2+ Kd1 36. Bf1 Rd2 37. Rd7 Rxd7 38. Bxc4 bxc4 39. Qxh8 Rd3 40. Qa8 c3?! { (2.61 → 3.63) Inaccuracy. Rb3+ was best. } (40... Rb3+ 41. Ka2 Rd3 42. Qa4+ Ke1 43. Qxc4 Rxf3 44. g4 Kf2 45. g5 Ke1 46. Qg4) 41. Qa4+ Ke1 42. f4 f5?! { (3.29 → 4.47) Inaccuracy. Rf3 was best. } (42... Rf3 43. Kc2 Rf2+ 44. Kxc3 Rxh2 45. g4 h5 46. g5 Rg2 47. Qe8+ Kf2 48. Qxf7) 43. Kc1 Rd2 44. Qa7 { White wins. } 1-0"""

def get_san_move(board, move):
    """Get the move in Standard Algebraic Notation (SAN) format"""
    return board.san(move)

def format_eval(cp_value):
    """Format centipawn evaluation to a readable string"""
    if cp_value is None:
        return "?"
    return f"{cp_value/100:+.2f}"

def get_colored_judgment(judgment):
    """Return the judgment with appropriate color based on type"""
    if judgment in JUDGMENT_COLORS:
        return f"{JUDGMENT_COLORS[judgment]}{judgment.value}{Style.RESET_ALL}"
    return str(judgment)

def print_analysis_table(game, evals, judgments):
    """Print the game analysis as a formatted table"""
    moves_list = []
    
    # Get the game mainline moves
    mainline_moves = list(game.mainline_moves())
    board = game.board()
    
    # Initialize variables to track move numbers
    move_num = 1
    is_white_move = True
    
    print("\n" + "="*80)
    print(f"{Fore.BLUE}{Style.BRIGHT}GAME ANALYSIS{Style.RESET_ALL}")
    print("="*80)
    
    # Loop through each move and evaluation
    for i, move in enumerate(mainline_moves):
        # Get move in SAN format
        san_move = get_san_move(board, move)
        
        # Get evaluations before and after the move
        prev_eval = format_eval(evals[i].cp) if i < len(evals) else "?"
        curr_eval = format_eval(evals[i+1].cp) if i+1 < len(evals) else "?"
        
        # Get judgment for the move
        judgment = judgments[i] if i < len(judgments) else None
        judgment_str = get_colored_judgment(judgment) if judgment else ""
        
        # Calculate evaluation change
        eval_change = ""
        if i+1 < len(evals) and evals[i].cp is not None and evals[i+1].cp is not None:
            change = (evals[i+1].cp - evals[i].cp) / 100
            # For Black's moves, negate the change
            if not is_white_move:
                change = -change
            
            # Format with +/- and color
            if change > 0:
                eval_change = f"{Fore.GREEN}+{change:.2f}{Style.RESET_ALL}"
            elif change < 0:
                eval_change = f"{Fore.RED}{change:.2f}{Style.RESET_ALL}"
            else:
                eval_change = f"{change:.2f}"
        
        # Add row to the table
        if is_white_move:
            moves_list.append([
                move_num,
                f"{san_move}",
                prev_eval,
                curr_eval,
                eval_change,
                judgment_str
            ])
        else:
            # Update the last row with black's move
            moves_list[-1].extend([
                f"{san_move}",
                prev_eval,
                curr_eval,
                eval_change,
                judgment_str
            ])
            move_num += 1
        
        # Push the move and toggle the active player
        board.push(move)
        is_white_move = not is_white_move
    
    # Create headers for the table
    headers = ["#", "White", "Eval Before", "Eval After", "Change", "Judgment",
               "Black", "Eval Before", "Eval After", "Change", "Judgment"]
    
    # Print the table
    print(tabulate(moves_list, headers=headers, tablefmt="pretty"))
    print("="*80)

def print_feature_summary(features):
    """Print the feature summary with improved formatting"""
    print("\n" + "="*80)
    print(f"{Fore.BLUE}{Style.BRIGHT}GAME FEATURE SUMMARY{Style.RESET_ALL}")
    print("="*80)
    
    # Organize features into categories
    categories = {
        "Game Phase": [
            "total_moves", "opening_length", "middlegame_length", "endgame_length"
        ],
        "Material/Position": [
            "material_balance_changes", "piece_mobility_avg", 
            "pawn_structure_changes", "center_control_avg"
        ],
        "White Move Quality": [
            "white_brilliant_count", "white_great_count", "white_good_moves",
            "white_inaccuracy_count", "white_mistake_count", "white_blunder_count",
            "white_avg_eval_change", "white_eval_volatility"
        ],
        "Black Move Quality": [
            "black_brilliant_count", "black_great_count", "black_good_moves",
            "black_inaccuracy_count", "black_mistake_count", "black_blunder_count",
            "black_avg_eval_change", "black_eval_volatility"
        ]
    }
    
    # Print each category of features
    for category, feature_names in categories.items():
        print(f"\n{Fore.CYAN}{Style.BRIGHT}{category}:{Style.RESET_ALL}")
        for name in feature_names:
            if name in features.__dict__:
                value = features.__dict__[name]
                print(f"  {name}: {value:.3f}")
    
    print("="*80)

# Main execution
print(f"{Fore.GREEN}Analyzing chess game...{Style.RESET_ALL}")

# Create a file-like object from the moves string
pgn_io = io.StringIO(moves)

# Parse the game from the PGN string
game = chess.pgn.read_game(pgn_io)

if game is None:
    print(f"{Fore.RED}Failed to parse game!{Style.RESET_ALL}")
else:
    print(f"{Fore.GREEN}Game parsed successfully!{Style.RESET_ALL}\n")
    
    # Initialize the feature extractor and the Stockfish handler
    feature_extractor = FeatureExtractor()
    stockfish = StockfishHandler(path="stockfish", depth=20)
    
    try:
        # Get positions from the game
        positions = feature_extractor._get_positions(game)
        
        # Evaluate each position using Stockfish
        print(f"{Fore.YELLOW}Evaluating positions with Stockfish...{Style.RESET_ALL}")
        evals = [stockfish.evaluate_position(pos, i) for i, pos in 
                 tqdm(enumerate(positions), total=len(positions), desc="Evaluating")]
        
        # Extract features
        features = feature_extractor.extract_features(game, evals)
        
        # Collect judgments from the analysis
        judgments = []
        mainline_moves = list(game.mainline_moves())
        
        # Collect move judgments
        for i in range(1, len(evals)):
            prev, curr = evals[i-1], evals[i]
            move_idx = i - 1
            
            if move_idx < len(mainline_moves):
                move = mainline_moves[move_idx]
                prev_board = positions[move_idx]
                curr_board = positions[move_idx + 1]
                
                judgment = MoveAnalyzer.analyze_move(
                    prev, curr, 
                    prev_board=prev_board, 
                    curr_board=curr_board, 
                    move=move
                )
                judgments.append(judgment)
        
        # Print the analysis table
        print_analysis_table(game, evals, judgments)
        
        # Print the feature summary
        print_feature_summary(features)
        
    except Exception as e:
        print(f"{Fore.RED}Error during analysis: {e}{Style.RESET_ALL}")
    finally:
        stockfish.close()

Analyzing chess game...
Game parsed successfully!

Evaluating positions with Stockfish...


Evaluating: 100%|██████████| 88/88 [02:09<00:00,  1.47s/it]



GAME ANALYSIS
+----+-------+-------------+------------+--------+------------+-------+-------------+------------+--------+------------+
| #  | White | Eval Before | Eval After | Change |  Judgment  | Black | Eval Before | Eval After | Change |  Judgment  |
+----+-------+-------------+------------+--------+------------+-------+-------------+------------+--------+------------+
| 1  |  e4   |    +0.24    |   +0.26    | +0.02  |    Good    |  d6   |    +0.26    |   +0.51    | -0.25  |    Good    |
| 2  |  d4   |    +0.51    |   +0.56    | +0.05  |    Good    |  Nf6  |    +0.56    |   +0.51    | +0.05  |    Good    |
| 3  |  Nc3  |    +0.51    |   +0.41    | -0.10  |    Good    |  g6   |    +0.41    |   +0.73    | -0.32  | Inaccuracy |
| 4  |  Be3  |    +0.73    |   +0.76    | +0.03  |    Good    |  Bg7  |    +0.76    |   +0.73    | +0.03  |    Good    |
| 5  |  Qd2  |    +0.73    |   +0.75    | +0.02  |    Good    |  c6   |    +0.75    |   +0.75    | -0.00  |    Good    |
| 6  |  f3   |   